In [1]:
import http.server
import socketserver
import webbrowser
import os

# --- Script Configuration ---
FILENAME = "interactive_presentation.html"
PORT = 8000
# --------------------------

# We define each slide's content as a separate HTML block in a list.
# This makes it easier to loop through and build the final file.
slides_content = [
    # Slide 1: Title
    """
    <div style="text-align: center;">
        <h1 style="font-size: 42px; font-weight: 700; color: #00529B; border-bottom: 2px solid #00529B; padding-bottom: 1rem; margin-bottom: 2rem;">Treasury Market: Weakening Foundations?</h1>
        <p style="font-size: 24px; color: #4b5563; margin-top: 1rem;">An Analysis of Supply, Demand, and Dealer Capacity</p>
        <p style="font-size: 20px; color: #6b7280; margin-top: 3rem;">Bob Sheehan, CFA, CMT</p>
        <p style="font-size: 16px; color: #9ca3af;">VP, Market Strategy & Research</p>
    </div>
    """,
    # Slide 2: Auction Weakness
    """
    <h2 class="slide-title">The Canary: A Tailed Auction</h2>
    <table class="slide-table">
        <tr>
            <td class="chart-cell">
                <img src="https://i.imgur.com/k9K2Y7l.png" alt="Bar chart showing auction buyer breakdown." class="chart-img">
            </td>
            <td class="text-cell">
                <p><strong>The Signal:</strong> The August 6th 10-year auction tailed by 1.1 basis points, the first tail in six months. The Treasury expected to pay 4.244% but had to offer 4.255% to clear the $42B auction.</p>
                <p><strong>Waning Demand:</strong> Foreign buyers (indirects) retreated to just 63.8% of the auction, well below the 70%+ average. This forced Primary Dealers to absorb 16.8% of the issue, a higher-than-average share.</p>
                <p><strong>A Persistent Trend:</strong> This isn't an isolated event. 45% of Treasury auctions in 2025 have tailed, suggesting a persistent weakness is emerging at the long end of the curve.</p>
            </td>
        </tr>
    </table>
    """,
    # Slide 3: Changing Buyer Base
    """
    <h2 class="slide-title">The Changing Buyer Base</h2>
    <table class="slide-table">
        <tr>
            <td class="chart-cell">
                <img src="https://i.imgur.com/oW5XyS3.png" alt="Stacked area chart of Treasury buyers." class="chart-img">
            </td>
            <td class="text-cell">
                <p><strong>The Shift:</strong> The composition of Treasury buyers has changed significantly since 2020. Foreign central banks have reduced their market share from 45% to just 28% today.</p>
                <p><strong>Who's Picking Up the Slack?</strong> Domestic banks have increased their share from 20% to 35%, and Primary Dealers have grown from 15% to 25%.</p>
                <p><strong>The Problem:</strong> These are not natural, long-term holders. They are intermediaries, not end-buyers, being forced to warehouse risk as traditional buyers retreat, creating a more fragile market foundation.</p>
            </td>
        </tr>
    </table>
    """,
    # Slide 4: Foreign Retreat
    """
    <h2 class="slide-title">Foreign Participation Declines</h2>
    <table class="slide-table">
        <tr>
            <td class="chart-cell">
                <img src="https://i.imgur.com/97yL2sU.png" alt="Line chart showing declining foreign participation." class="chart-img">
            </td>
            <td class="chart-cell">
                <img src="https://i.imgur.com/Q2yVf2Q.png" alt="Line chart showing dealer absorption." class="chart-img">
            </td>
        </tr>
    </table>
    """,
    # Slide 5: Gross vs Net
    """
    <h2 class="slide-title">Dealer Inventory: Gross vs. Net</h2>
    <table class="slide-table">
        <tr>
            <td class="chart-cell">
                <img src="https://i.imgur.com/j8n6nE1.png" alt="Line chart of Gross vs Net dealer inventory." class="chart-img">
            </td>
            <td class="text-cell">
                <p><strong>Gross Inventory Surges:</strong> The red line shows dealers' gross Treasury inventory has surged to ~$120B, approaching the highs seen before the September 2019 repo crisis. This represents the total collateral they must finance daily.</p>
                <p><strong>Net Position is Flat:</strong> The blue line shows their net position (longs minus shorts) is nearly flat at -$0.3B. This indicates dealers are well-hedged and not making a large directional bet on rates.</p>
                <p><strong>The Real Risk:</strong> The divergence is key. The risk isn't directional loss; it's a mechanical funding risk from the massive gross inventory, making the system vulnerable to a funding squeeze.</p>
            </td>
        </tr>
    </table>
    """,
    # Slide 6: Accumulation Momentum
    """
    <h2 class="slide-title">Dealer Accumulation Momentum</h2>
    <table class="slide-table">
        <tr>
            <td class="chart-cell">
                <img src="https://i.imgur.com/39J6i5Q.png" alt="Area chart of dealer inventory accumulation momentum." class="chart-img">
            </td>
            <td class="text-cell">
                <p><strong>Measuring the Flow:</strong> This chart shows the momentum of dealer inventory accumulation by comparing the 26-week moving average to the 52-week moving average.</p>
                <p><strong>Rapid Accumulation:</strong> The pink zones highlight periods where the short-term trend exceeds the long-term trend. We are currently in such a period, indicating dealers are absorbing bonds faster than they can distribute them.</p>
                <p><strong>A Leading Indicator:</strong> This momentum view reveals hidden stress. Sustained periods of rapid accumulation, like in 2024-25, have historically preceded periods of market instability.</p>
            </td>
        </tr>
    </table>
    """,
    # Slide 7: Stablecoin Demand
    """
    <h2 class="slide-title">The Stablecoin Factor: A New Source of Demand</h2>
    <table class="slide-table">
        <tr>
            <td class="chart-cell">
                <img src="https://i.imgur.com/pA11zNn.png" alt="Line chart of stablecoin T-Bill holdings." class="chart-img">
            </td>
            <td class="text-cell">
                <p><strong>A Concentrated Buyer:</strong> A new, significant buyer has emerged at the front-end of the curve. Stablecoin issuers now hold over $180B in T-Bills, with Tether alone holding $125B.</p>
                <p><strong>Regulatory Arbitrage?</strong> These entities must back their tokens with ultra-safe, liquid assets, creating a massive, concentrated demand for T-bills (typically 90 days or less).</p>
                <p><strong>Fragile Demand:</strong> This demand is not fundamentally driven; it's a function of the crypto market. A reversal in crypto or a regulatory crackdown could cause this significant source of demand to vanish quickly.</p>
            </td>
        </tr>
    </table>
    """,
    # Slide 8: Bills vs Bonds
    """
    <h2 class="slide-title">A Tale of Two Markets: Bills vs. Bonds</h2>
    <table class="slide-table">
        <tr>
            <td class="chart-cell"><img src="https://i.imgur.com/dK5uL9S.png" alt="Line chart of T-Bill auction demand." class="chart-img"></td>
            <td class="chart-cell"><img src="https://i.imgur.com/Qz7L74U.png" alt="Line chart of 10-Year auction demand." class="chart-img"></td>
        </tr>
        <tr>
            <td colspan="2" style="font-size: 15px; line-height: 1.6; text-align: center; padding-top: 15px;">
                <p>A clear divergence: Demand for short-term T-Bills is robust, while demand for long-term bonds is weakening, shown by falling bid-to-cover ratios in 10-year auctions.</p>
            </td>
        </tr>
    </table>
    """,
    # Slide 9: Supply Surge
    """
    <h2 class="slide-title">The Oncoming Supply Wave</h2>
    <table class="slide-table">
        <tr>
            <td class="chart-cell">
                <img src="https://i.imgur.com/N71S9iL.png" alt="Bar chart of increasing quarterly Treasury issuance." class="chart-img">
            </td>
            <td class="text-cell">
                <p><strong>Issuance is Accelerating:</strong> The supply of new Treasury debt is compounding the demand problem. Net issuance is projected to hit $1.5T in the second half of 2025 alone.</p>
                <p><strong>Poor Timing:</strong> This surge in supply is arriving at the precise moment when traditional foreign buyers are retreating and intermediary dealers are hitting their balance sheet constraints.</p>
            </td>
        </tr>
    </table>
    """,
    # Slide 10: Stress Scenarios
    """
    <h2 class="slide-title">Potential Market Impact Under Stress</h2>
    <table class="slide-table">
        <tr>
            <td class="chart-cell">
                <img src="https://i.imgur.com/95oFhM7.png" alt="Bar chart of stress scenario impacts." class="chart-img">
            </td>
            <td class="text-cell">
                <p><strong>Quantifying the Risk:</strong> This chart models the potential market impact as dealer inventory (a proxy for market stress) increases.</p>
                <p><strong>Moderate Stress ($150B):</strong> If foreign demand continues to decline, 10-year yields could reach 4.75% - 5.00%.</p>
                <p><strong>Severe Stress ($200B+):</strong> A severe scenario could push yields above 5% and lead to a violent bear-steepening of the yield curve, with significant negative consequences for risk assets.</p>
            </td>
        </tr>
    </table>
    """,
    # Slide 11: Stress Dashboard
    """
    <h2 class="slide-title">Treasury Market Stress Dashboard</h2>
    <table style="width:100%; height:80%; border-spacing: 15px; border-collapse: separate;">
        <tr>
            <td class="chart-cell"><img src="https://i.imgur.com/hXqYj6R.png" alt="Auction Tails chart." class="chart-img"></td>
            <td class="chart-cell"><img src="https://i.imgur.com/5V44r4l.png" alt="Foreign Demand chart." class="chart-img"></td>
        </tr>
        <tr>
            <td class="chart-cell"><img src="https://i.imgur.com/jM8V8Wz.png" alt="Dealer Capacity chart." class="chart-img"></td>
            <td class="chart-cell"><img src="https://i.imgur.com/8Qp2F6T.png" alt="Funding Stress chart." class="chart-img"></td>
        </tr>
    </table>
    """,
    # Slide 12: Z-Score
    """
    <h2 class="slide-title">Dealer Balance Sheet Stress (Z-Score)</h2>
    <table class="slide-table">
        <tr>
            <td class="chart-cell"><img src="https://i.imgur.com/zWwYd6V.png" alt="Line chart of Dealer Position Z-Score." class="chart-img"></td>
            <td class="text-cell">
                <p><strong>Normalizing the Data:</strong> This chart shows the Z-score of dealer net Treasury positions, measuring how many standard deviations the current position is from its historical average.</p>
                <p><strong>Interpreting the Signal:</strong> Values between +2 and -2 are considered normal. Spikes beyond indicate significant stress.</p>
                <p><strong>Current Reading:</strong> While there have been past spikes, the current Z-score is well within the normal range, indicating that on a statistically normalized basis, dealer balance sheets are not yet at a breaking point.</p>
            </td>
        </tr>
    </table>
    """,
    # Slide 13: Rolling Volatility
    """
    <h2 class="slide-title">Rolling Volatility of Net Positions</h2>
    <table class="slide-table">
        <tr>
            <td class="chart-cell"><img src="https://i.imgur.com/8QG2V25.png" alt="Line chart of rolling volatility." class="chart-img"></td>
            <td class="text-cell">
                <p><strong>Measuring Instability:</strong> This chart tracks the rolling 13-week (short-term) and 52-week (long-term) volatility of dealer net Treasury positions.</p>
                <p><strong>Spikes in Volatility:</strong> Periods where short-term vol (purple) spikes above long-term vol (grey) indicate market instability.</p>
                <p><strong>Current Reading:</strong> Short-term volatility is elevated and has crossed above the long-term trend, suggesting the *rate of change* in positioning is becoming erratic, which can be a precursor to broader market volatility.</p>
            </td>
        </tr>
    </table>
    """,
    # Slide 14: Conclusion
    """
    <h2 class="slide-title">Conclusion & Key Takeaways</h2>
    <div style="font-size: 18px; line-height: 1.8; color: #374151; max-width: 800px; margin: 0 auto;">
        <ul style="list-style-type: disc; padding-left: 30px; text-align: left;">
            <li style="margin-bottom: 20px;"><strong>Weakening Foundations:</strong> Multiple data points indicate a structural weakening in the demand for long-term U.S. Treasury debt.</li>
            <li style="margin-bottom: 20px;"><strong>Dealer Constraints are Key:</strong> Primary dealers, the market's backstop, are absorbing massive duration and approaching their regulatory limits.</li>
            <li style="margin-bottom: 20px;"><strong>Supply is a Major Headwind:</strong> An accelerating wave of Treasury supply is set to hit the market just as demand is faltering and intermediaries are constrained.</li>
            <li style="margin-bottom: 20px;"><strong>Increased Risk of a "Gap Higher" in Yields:</strong> The combination of these factors increases the probability of a disorderly move higher in long-term interest rates.</li>
        </ul>
    </div>
    """
]

# Build the final HTML for all slides
final_slides_html = ""
for i, content in enumerate(slides_content):
    # The first slide is active by default, others are hidden.
    active_class = "active" if i == 0 else ""
    final_slides_html += f'<section class="slide {active_class}">{content}</section>\n'

# Create the full HTML document with embedded CSS and JavaScript
full_html_doc = f"""
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Presentation Preview</title>
    <style>
        body {{
            font-family: 'Inter', Arial, sans-serif;
            background-color: #374151; /* gray-700 */
            margin: 0;
            display: flex;
            justify-content: center;
            align-items: center;
            height: 100vh;
        }}
        #presentation-container {{
            width: 100%;
            max-width: 1200px;
            height: 90vh;
            max-height: 800px;
            position: relative;
            box-shadow: 0 10px 25px rgba(0,0,0,0.2);
        }}
        .slide {{
            display: none; /* Hide all slides by default */
            width: 100%;
            height: 100%;
            padding: 3rem;
            background-color: #FFFFFF;
            color: #1F2937;
            flex-direction: column;
            justify-content: center;
            align-items: center;
            box-sizing: border-box;
            position: relative;
        }}
        .slide.active {{
            display: flex; /* Show the active slide */
        }}
        .slide-title {{
            font-size: 32px;
            font-weight: 700;
            color: #00529B;
            border-bottom: 2px solid #00529B;
            padding-bottom: 0.5rem;
            margin-bottom: 2rem;
            position: absolute;
            top: 3rem;
            left: 3rem;
            right: 3rem;
            text-align: left;
        }}
        .slide-table {{ width: 100%; height: 80%; border-spacing: 20px; border-collapse: separate; margin-top: 4rem; }}
        .chart-cell {{ width: 50%; vertical-align: middle; text-align: center; }}
        .text-cell {{ width: 50%; vertical-align: middle; font-size: 16px; line-height: 1.6; padding: 0 1rem; }}
        .text-cell p {{ margin-bottom: 1.5em; }}
        .chart-img {{ max-width: 100%; max-height: 350px; height: auto; border: 1px solid #e5e7eb; }}
        #nav-controls {{
            position: absolute;
            bottom: 20px;
            right: 20px;
            display: flex;
            gap: 10px;
        }}
        .nav-btn {{
            background-color: #4B5563;
            color: white;
            border: none;
            border-radius: 50%;
            width: 50px;
            height: 50px;
            font-size: 24px;
            font-weight: bold;
            cursor: pointer;
            box-shadow: 0 4px 6px rgba(0,0,0,0.1);
            transition: background-color 0.2s;
        }}
        .nav-btn:hover {{ background-color: #00529B; }}
        .slide-footer {{
            position: absolute;
            bottom: 3rem;
            left: 3rem;
            right: 3rem;
            display: flex;
            justify-content: space-between;
            font-size: 14px;
            color: #6B7280;
        }}
        .logo {{ font-weight: 700; color: #00529B; }}
    </style>
</head>
<body>
    <div id="presentation-container">
        {final_slides_html}
        <div id="slide-counter" class="slide-footer"></div>
    </div>
    <div id="nav-controls">

SyntaxError: incomplete input (ipython-input-3213620170.py, line 227)

In [ ]:
import http.server
import socketserver
import webbrowser
import os

# --- Script Configuration ---
FILENAME = "interactive_presentation.html"
PORT = 8000
# --------------------------

# We define each slide's content as a separate HTML block in a list.
# This makes it easier to loop through and build the final file.
slides_content = [
    # Slide 1: Title
    """
    <div style="text-align: center;">
        <h1 style="font-size: 42px; font-weight: 700; color: #00529B; border-bottom: 2px solid #00529B; padding-bottom: 1rem; margin-bottom: 2rem;">Treasury Market: Weakening Foundations?</h1>
        <p style="font-size: 24px; color: #4b5563; margin-top: 1rem;">An Analysis of Supply, Demand, and Dealer Capacity</p>
        <p style="font-size: 20px; color: #6b7280; margin-top: 3rem;">Bob Sheehan, CFA, CMT</p>
        <p style="font-size: 16px; color: #9ca3af;">VP, Market Strategy & Research</p>
    </div>
    """,
    # Slide 2: Auction Weakness
    """
    <h2 class="slide-title">The Canary: A Tailed Auction</h2>
    <table class="slide-table">
        <tr>
            <td class="chart-cell">
                <img src="https://i.imgur.com/k9K2Y7l.png" alt="Bar chart showing auction buyer breakdown." class="chart-img">
            </td>
            <td class="text-cell">
                <p><strong>The Signal:</strong> The August 6th 10-year auction tailed by 1.1 basis points, the first tail in six months. The Treasury expected to pay 4.244% but had to offer 4.255% to clear the $42B auction.</p>
                <p><strong>Waning Demand:</strong> Foreign buyers (indirects) retreated to just 63.8% of the auction, well below the 70%+ average. This forced Primary Dealers to absorb 16.8% of the issue, a higher-than-average share.</p>
                <p><strong>A Persistent Trend:</strong> This isn't an isolated event. 45% of Treasury auctions in 2025 have tailed, suggesting a persistent weakness is emerging at the long end of the curve.</p>
            </td>
        </tr>
    </table>
    """,
    # Slide 3: Changing Buyer Base
    """
    <h2 class="slide-title">The Changing Buyer Base</h2>
    <table class="slide-table">
        <tr>
            <td class="chart-cell">
                <img src="https://i.imgur.com/oW5XyS3.png" alt="Stacked area chart of Treasury buyers." class="chart-img">
            </td>
            <td class="text-cell">
                <p><strong>The Shift:</strong> The composition of Treasury buyers has changed significantly since 2020. Foreign central banks have reduced their market share from 45% to just 28% today.</p>
                <p><strong>Who's Picking Up the Slack?</strong> Domestic banks have increased their share from 20% to 35%, and Primary Dealers have grown from 15% to 25%.</p>
                <p><strong>The Problem:</strong> These are not natural, long-term holders. They are intermediaries, not end-buyers, being forced to warehouse risk as traditional buyers retreat, creating a more fragile market foundation.</p>
            </td>
        </tr>
    </table>
    """,
    # Slide 4: Foreign Retreat
    """
    <h2 class="slide-title">Foreign Participation Declines</h2>
    <table class="slide-table">
        <tr>
            <td class="chart-cell">
                <img src="https://i.imgur.com/97yL2sU.png" alt="Line chart showing declining foreign participation." class="chart-img">
            </td>
            <td class="chart-cell">
                <img src="https://i.imgur.com/Q2yVf2Q.png" alt="Line chart showing dealer absorption." class="chart-img">
            </td>
        </tr>
    </table>
    """,
    # Slide 5: Gross vs Net
    """
    <h2 class="slide-title">Dealer Inventory: Gross vs. Net</h2>
    <table class="slide-table">
        <tr>
            <td class="chart-cell">
                <img src="https://i.imgur.com/j8n6nE1.png" alt="Line chart of Gross vs Net dealer inventory." class="chart-img">
            </td>
            <td class="text-cell">
                <p><strong>Gross Inventory Surges:</strong> The red line shows dealers' gross Treasury inventory has surged to ~$120B, approaching the highs seen before the September 2019 repo crisis. This represents the total collateral they must finance daily.</p>
                <p><strong>Net Position is Flat:</strong> The blue line shows their net position (longs minus shorts) is nearly flat at -$0.3B. This indicates dealers are well-hedged and not making a large directional bet on rates.</p>
                <p><strong>The Real Risk:</strong> The divergence is key. The risk isn't directional loss; it's a mechanical funding risk from the massive gross inventory, making the system vulnerable to a funding squeeze.</p>
            </td>
        </tr>
    </table>
    """,
    # Slide 6: Accumulation Momentum
    """
    <h2 class="slide-title">Dealer Accumulation Momentum</h2>
    <table class="slide-table">
        <tr>
            <td class="chart-cell">
                <img src="https://i.imgur.com/39J6i5Q.png" alt="Area chart of dealer inventory accumulation momentum." class="chart-img">
            </td>
            <td class="text-cell">
                <p><strong>Measuring the Flow:</strong> This chart shows the momentum of dealer inventory accumulation by comparing the 26-week moving average to the 52-week moving average.</p>
                <p><strong>Rapid Accumulation:</strong> The pink zones highlight periods where the short-term trend exceeds the long-term trend. We are currently in such a period, indicating dealers are absorbing bonds faster than they can distribute them.</p>
                <p><strong>A Leading Indicator:</strong> This momentum view reveals hidden stress. Sustained periods of rapid accumulation, like in 2024-25, have historically preceded periods of market instability.</p>
            </td>
        </tr>
    </table>
    """,
    # Slide 7: Stablecoin Demand
    """
    <h2 class="slide-title">The Stablecoin Factor: A New Source of Demand</h2>
    <table class="slide-table">
        <tr>
            <td class="chart-cell">
                <img src="https://i.imgur.com/pA11zNn.png" alt="Line chart of stablecoin T-Bill holdings." class="chart-img">
            </td>
            <td class="text-cell">
                <p><strong>A Concentrated Buyer:</strong> A new, significant buyer has emerged at the front-end of the curve. Stablecoin issuers now hold over $180B in T-Bills, with Tether alone holding $125B.</p>
                <p><strong>Regulatory Arbitrage?</strong> These entities must back their tokens with ultra-safe, liquid assets, creating a massive, concentrated demand for T-bills (typically 90 days or less).</p>
                <p><strong>Fragile Demand:</strong> This demand is not fundamentally driven; it's a function of the crypto market. A reversal in crypto or a regulatory crackdown could cause this significant source of demand to vanish quickly.</p>
            </td>
        </tr>
    </table>
    """,
    # Slide 8: Bills vs Bonds
    """
    <h2 class="slide-title">A Tale of Two Markets: Bills vs. Bonds</h2>
    <table class="slide-table">
        <tr>
            <td class="chart-cell"><img src="https://i.imgur.com/dK5uL9S.png" alt="Line chart of T-Bill auction demand." class="chart-img"></td>
            <td class="chart-cell"><img src="https://i.imgur.com/Qz7L74U.png" alt="Line chart of 10-Year auction demand." class="chart-img"></td>
        </tr>
        <tr>
            <td colspan="2" style="font-size: 15px; line-height: 1.6; text-align: center; padding-top: 15px;">
                <p>A clear divergence: Demand for short-term T-Bills is robust, while demand for long-term bonds is weakening, shown by falling bid-to-cover ratios in 10-year auctions.</p>
            </td>
        </tr>
    </table>
    """,
    # Slide 9: Supply Surge
    """
    <h2 class="slide-title">The Oncoming Supply Wave</h2>
    <table class="slide-table">
        <tr>
            <td class="chart-cell">
                <img src="https://i.imgur.com/N71S9iL.png" alt="Bar chart of increasing quarterly Treasury issuance." class="chart-img">
            </td>
            <td class="text-cell">
                <p><strong>Issuance is Accelerating:</strong> The supply of new Treasury debt is compounding the demand problem. Net issuance is projected to hit $1.5T in the second half of 2025 alone.</p>
                <p><strong>Poor Timing:</strong> This surge in supply is arriving at the precise moment when traditional foreign buyers are retreating and intermediary dealers are hitting their balance sheet constraints.</p>
            </td>
        </tr>
    </table>
    """,
    # Slide 10: Stress Scenarios
    """
    <h2 class="slide-title">Potential Market Impact Under Stress</h2>
    <table class="slide-table">
        <tr>
            <td class="chart-cell">
                <img src="https://i.imgur.com/95oFhM7.png" alt="Bar chart of stress scenario impacts." class="chart-img">
            </td>
            <td class="text-cell">
                <p><strong>Quantifying the Risk:</strong> This chart models the potential market impact as dealer inventory (a proxy for market stress) increases.</p>
                <p><strong>Moderate Stress ($150B):</strong> If foreign demand continues to decline, 10-year yields could reach 4.75% - 5.00%.</p>
                <p><strong>Severe Stress ($200B+):</strong> A severe scenario could push yields above 5% and lead to a violent bear-steepening of the yield curve, with significant negative consequences for risk assets.</p>
            </td>
        </tr>
    </table>
    """,
    # Slide 11: Stress Dashboard
    """
    <h2 class="slide-title">Treasury Market Stress Dashboard</h2>
    <table style="width:100%; height:80%; border-spacing: 15px; border-collapse: separate;">
        <tr>
            <td class="chart-cell"><img src="https://i.imgur.com/hXqYj6R.png" alt="Auction Tails chart." class="chart-img"></td>
            <td class="chart-cell"><img src="https://i.imgur.com/5V44r4l.png" alt="Foreign Demand chart." class="chart-img"></td>
        </tr>
        <tr>
            <td class="chart-cell"><img src="https://i.imgur.com/jM8V8Wz.png" alt="Dealer Capacity chart." class="chart-img"></td>
            <td class="chart-cell"><img src="https://i.imgur.com/8Qp2F6T.png" alt="Funding Stress chart." class="chart-img"></td>
        </tr>
    </table>
    """,
    # Slide 12: Z-Score
    """
    <h2 class="slide-title">Dealer Balance Sheet Stress (Z-Score)</h2>
    <table class="slide-table">
        <tr>
            <td class="chart-cell"><img src="https://i.imgur.com/zWwYd6V.png" alt="Line chart of Dealer Position Z-Score." class="chart-img"></td>
            <td class="text-cell">
                <p><strong>Normalizing the Data:</strong> This chart shows the Z-score of dealer net Treasury positions, measuring how many standard deviations the current position is from its historical average.</p>
                <p><strong>Interpreting the Signal:</strong> Values between +2 and -2 are considered normal. Spikes beyond indicate significant stress.</p>
                <p><strong>Current Reading:</strong> While there have been past spikes, the current Z-score is well within the normal range, indicating that on a statistically normalized basis, dealer balance sheets are not yet at a breaking point.</p>
            </td>
        </tr>
    </table>
    """,
    # Slide 13: Rolling Volatility
    """
    <h2 class="slide-title">Rolling Volatility of Net Positions</h2>
    <table class="slide-table">
        <tr>
            <td class="chart-cell"><img src="https://i.imgur.com/8QG2V25.png" alt="Line chart of rolling volatility." class="chart-img"></td>
            <td class="text-cell">
                <p><strong>Measuring Instability:</strong> This chart tracks the rolling 13-week (short-term) and 52-week (long-term) volatility of dealer net Treasury positions.</p>
                <p><strong>Spikes in Volatility:</strong> Periods where short-term vol (purple) spikes above long-term vol (grey) indicate market instability.</p>
                <p><strong>Current Reading:</strong> Short-term volatility is elevated and has crossed above the long-term trend, suggesting the *rate of change* in positioning is becoming erratic, which can be a precursor to broader market volatility.</p>
            </td>
        </tr>
    </table>
    """,
    # Slide 14: Conclusion
    """
    <h2 class="slide-title">Conclusion & Key Takeaways</h2>
    <div style="font-size: 18px; line-height: 1.8; color: #374151; max-width: 800px; margin: 0 auto;">
        <ul style="list-style-type: disc; padding-left: 30px; text-align: left;">
            <li style="margin-bottom: 20px;"><strong>Weakening Foundations:</strong> Multiple data points indicate a structural weakening in the demand for long-term U.S. Treasury debt.</li>
            <li style="margin-bottom: 20px;"><strong>Dealer Constraints are Key:</strong> Primary dealers, the market's backstop, are absorbing massive duration and approaching their regulatory limits.</li>
            <li style="margin-bottom: 20px;"><strong>Supply is a Major Headwind:</strong> An accelerating wave of Treasury supply is set to hit the market just as demand is faltering and intermediaries are constrained.</li>
            <li style="margin-bottom: 20px;"><strong>Increased Risk of a "Gap Higher" in Yields:</strong> The combination of these factors increases the probability of a disorderly move higher in long-term interest rates.</li>
        </ul>
    </div>
    """
]

# Build the final HTML for all slides
final_slides_html = ""
for i, content in enumerate(slides_content):
    # The first slide is active by default, others are hidden.
    active_class = "active" if i == 0 else ""
    final_slides_html += f'<section class="slide {active_class}">{content}</section>\n'

# Create the full HTML document with embedded CSS and JavaScript
full_html_doc = f"""
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Presentation Preview</title>
    <style>
        body {{
            font-family: 'Inter', Arial, sans-serif;
            background-color: #374151; /* gray-700 */
            margin: 0;
            display: flex;
            justify-content: center;
            align-items: center;
            height: 100vh;
        }}
        #presentation-container {{
            width: 100%;
            max-width: 1200px;
            height: 90vh;
            max-height: 800px;
            position: relative;
            box-shadow: 0 10px 25px rgba(0,0,0,0.2);
        }}
        .slide {{
            display: none; /* Hide all slides by default */
            width: 100%;
            height: 100%;
            padding: 3rem;
            background-color: #FFFFFF;
            color: #1F2937;
            flex-direction: column;
            justify-content: center;
            align-items: center;
            box-sizing: border-box;
            position: relative;
        }}
        .slide.active {{
            display: flex; /* Show the active slide */
        }}
        .slide-title {{
            font-size: 32px;
            font-weight: 700;
            color: #00529B;
            border-bottom: 2px solid #00529B;
            padding-bottom: 0.5rem;
            margin-bottom: 2rem;
            position: absolute;
            top: 3rem;
            left: 3rem;
            right: 3rem;
            text-align: left;
        }}
        .slide-table {{ width: 100%; height: 80%; border-spacing: 20px; border-collapse: separate; margin-top: 4rem; }}
        .chart-cell {{ width: 50%; vertical-align: middle; text-align: center; }}
        .text-cell {{ width: 50%; vertical-align: middle; font-size: 16px; line-height: 1.6; padding: 0 1rem; }}
        .text-cell p {{ margin-bottom: 1.5em; }}
        .chart-img {{ max-width: 100%; max-height: 350px; height: auto; border: 1px solid #e5e7eb; }}
        #nav-controls {{
            position: absolute;
            bottom: 20px;
            right: 20px;
            display: flex;
            gap: 10px;
        }}
        .nav-btn {{
            background-color: #4B5563;
            color: white;
            border: none;
            border-radius: 50%;
            width: 50px;
            height: 50px;
            font-size: 24px;
            font-weight: bold;
            cursor: pointer;
            box-shadow: 0 4px 6px rgba(0,0,0,0.1);
            transition: background-color 0.2s;
        }}
        .nav-btn:hover {{ background-color: #00529B; }}
        .slide-footer {{
            position: absolute;
            bottom: 3rem;
            left: 3rem;
            right: 3rem;
            display: flex;
            justify-content: space-between;
            font-size: 14px;
            color: #6B7280;
        }}
        .logo {{ font-weight: 700; color: #00529B; }}
    </style>
</head>
<body>
    <div id="presentation-container">
        {final_slides_html}
        <div id="slide-counter" class="slide-footer"></div>
    </div>
    <div id="nav-controls">
        <button id="prevBtn" class="nav-btn">&lt;</button>
        <button id="nextBtn" class="nav-btn">&gt;</button>
    </div>

    <script>
        const slides = document.querySelectorAll('.slide');
        const slideCounter = document.getElementById('slide-counter');
        const totalSlides = slides.length;
        let currentSlide = 0;

        function showSlide(index) {{
            // Ensure index is within bounds
            currentSlide = (index + totalSlides) % totalSlides;

            slides.forEach((slide, i) => {{
                slide.classList.toggle('active', i === currentSlide);
            }});

            // Update footer
            const footer = slides[currentSlide].querySelector('.slide-footer') || document.createElement('div');
            if (!slides[currentSlide].contains(footer)) {{
                 slides[currentSlide].appendChild(footer);
            }}
            footer.innerHTML = `
                <div class="logo">EquiLend Data & Analytics</div>
                <div>${{currentSlide + 1}} / ${{totalSlides}}</div>
            `;
        }}

        document.getElementById('nextBtn').addEventListener('click', () => {{
            showSlide(currentSlide + 1);
        }});

        document.getElementById('prevBtn').addEventListener('click', () => {{
            showSlide(currentSlide - 1);
        }});

        // Keyboard navigation
        document.addEventListener('keydown', (e) => {{
            if (e.key === 'ArrowRight') {{
                showSlide(currentSlide + 1);
            }} else if (e.key === 'ArrowLeft') {{
                showSlide(currentSlide - 1);
            }}
        }});

        // Show the first slide initially
        showSlide(0);
    </script>
</body>
</html>
"""

# --- Main script execution ---

# 1. Write the final HTML to a file
try:
    with open(FILENAME, "w", encoding="utf-8") as f:
        f.write(full_html_doc)
    print(f"✅ Successfully created interactive presentation: '{FILENAME}'")
except IOError as e:
    print(f"❌ Error writing file: {e}")
    exit()

# 2. Set up and run the local server
Handler = http.server.SimpleHTTPRequestHandler
httpd = None
try:
    httpd = socketserver.TCPServer(("", PORT), Handler)
except OSError:
    print(f"❌ Port {PORT} is already in use. Please close the other program or change the PORT number in the script.")
    exit()

# 3. Open the web browser to the file
url = f"http://localhost:{PORT}/{FILENAME}"
print(f"🚀 Starting server at: {url}")
print("   Navigate with arrow keys or on-screen buttons.")
print("   To stop the server, press Ctrl+C in this terminal.")
webbrowser.open_new_tab(url)

# 4. Start the server's main loop
try:
    httpd.serve_forever()
except KeyboardInterrupt:
    print("\n✅ Server stopped.")
    httpd.server_close()

✅ Successfully created interactive presentation: 'interactive_presentation.html'
🚀 Starting server at: http://localhost:8000/interactive_presentation.html
   Navigate with arrow keys or on-screen buttons.
   To stop the server, press Ctrl+C in this terminal.


In [ ]:
import http.server
import socketserver
import webbrowser
import os
import base64
import requests # Make sure you have installed this library

# --- Script Configuration ---
FILENAME = "treasury_foundations_presentation_full.html"
PORT = 8000
# --------------------------

# Dictionary of all 15 chart images to use from local files.
chart_files = {
    "auction_analysis": "/content/unnamed (6).png",
    "buyer_composition": "/content/unnamed (5).png",
    "foreign_participation": "/content/unnamed (4).png", # Left side of the dual chart
    "dealer_absorption": "/content/unnamed (4).png", # Right side of the dual chart
    "gross_vs_net": "/content/unnamed (3).jpg",
    "accumulation_momentum": "/content/unnamed (1).png",
    "stablecoin_holdings": "/content/unnamed (3).png",
    "bills_vs_bonds": "/content/unnamed (2).png",
    "supply_surge": "/content/t1.jpg",
    "stress_scenarios": "/content/unnamed (8).png",
    "stress_dashboard": "/content/unnamed.jpg",
    "z_score": "/content/unnamed (1).jpg",
    "rolling_volatility": "/content/unnamed (2).jpg",
    "position_changes": "/content/unnamed.png", # This seems to be a duplicate of accumulation, will use for context
    "new_chart": "/content/unnamed (7).png" # Added the new chart
}

def get_image_as_base64_from_file(filepath):
    """Reads an image file and returns it as a Base64 encoded string."""
    try:
        with open(filepath, "rb") as image_file:
            encoded_string = base64.b64encode(image_file.read()).decode('utf-8')
            # The mime type can be derived from the file extension or a library if needed, assuming png/jpg for simplicity
            # A more robust solution would inspect the file content
            if filepath.lower().endswith('.png'):
                return f"data:image/png;base64,{encoded_string}"
            elif filepath.lower().endswith('.jpg') or filepath.lower().endswith('.jpeg'):
                 return f"data:image/jpeg;base64,{encoded_string}"
            else:
                print(f"Warning: Could not determine mime type for {filepath}. Assuming png.")
                return f"data:image/png;base64,{encoded_string}"
    except IOError as e:
        print(f"❌ Error reading file {filepath}: {e}")
        return "" # Return an empty string if the file cannot be read

print("Reading and embedding chart images from local files...")
base64_images = {name: get_image_as_base64_from_file(filepath) for name, filepath in chart_files.items()}
print("✅ All charts ready.")


# The slide content now uses placeholders like {slide2_chart} for the image sources
slides_content = [
    # Slide 1: Title
    """
    <div style="text-align: center;">
        <h1 style="font-size: 42px; font-weight: 700; color: #00529B; border-bottom: 2px solid #00529B; padding-bottom: 1rem; margin-bottom: 2rem;">Treasury Market: Weakening Foundations?</h1>
        <p style="font-size: 24px; color: #4b5563; margin-top: 1rem;">An Analysis of Supply, Demand, and Dealer Capacity</p>
        <p style="font-size: 20px; color: #6b7280; margin-top: 3rem;">Bob Sheehan, CFA, CMT</p>
        <p style="font-size: 16px; color: #9ca3af;">VP, Market Strategy & Research</p>
    </div>
    """,
    # Slide 2: Auction Weakness
    """
    <h2 class="slide-title">The Canary: A Tailed Auction</h2>
    <table class="slide-table"><tr><td class="chart-cell"><img src="{auction_analysis}" alt="Bar chart showing auction buyer breakdown." class="chart-img"></td><td class="text-cell"><p><strong>The Signal:</strong> The August 6th 10-year auction tailed by 1.1 basis points, the first tail in six months. The Treasury expected to pay 4.244% but had to offer 4.255% to clear the $42B auction.</p><p><strong>Waning Demand:</strong> Foreign buyers (indirects) retreated to just 63.8% of the auction, well below the 70%+ average. This forced Primary Dealers to absorb 16.8% of the issue.</p><p><strong>A Persistent Trend:</strong> 45% of Treasury auctions in 2025 have tailed, suggesting a persistent weakness is emerging at the long end of the curve.</p></td></tr></table>
    """.format(**base64_images),
    # Slide 3: Changing Buyer Base
    """
    <h2 class="slide-title">The Changing Buyer Base</h2>
    <table class="slide-table"><tr><td class="chart-cell"><img src="{buyer_composition}" alt="Stacked area chart of Treasury buyers." class="chart-img"></td><td class="text-cell"><p><strong>The Shift:</strong> The composition of Treasury buyers has changed significantly since 2020. Foreign central banks have reduced their market share from 45% to just 28% today.</p><p><strong>Who's Picking Up the Slack?</strong> Domestic banks have increased their share from 20% to 35%, and Primary Dealers have grown from 15% to 25%.</p><p><strong>The Problem:</strong> These are not natural, long-term holders. They are intermediaries being forced to warehouse risk as traditional buyers retreat.</p></td></tr></table>
    """.format(**base64_images),
    # Slide 4: Foreign Retreat
    """
    <h2 class="slide-title">Foreign Participation Declines</h2>
    <table class="slide-table"><tr><td class="chart-cell"><img src="{foreign_participation}" alt="Line chart showing declining foreign participation." class="chart-img"></td><td class="chart-cell"><img src="{dealer_absorption}" alt="Line chart showing dealer absorption." class="chart-img"></td></tr></table>
    """.format(**base64_images),
    # Slide 5: Gross vs Net
    """
    <h2 class="slide-title">Dealer Inventory: Gross vs. Net</h2>
    <table class="slide-table"><tr><td class="chart-cell"><img src="{gross_vs_net}" alt="Line chart of Gross vs Net dealer inventory." class="chart-img"></td><td class="text-cell"><p><strong>Gross Inventory Surges:</strong> Dealers' gross Treasury inventory has surged to ~$120B, approaching highs seen before the Sep. 2019 repo crisis. This is the collateral they must finance daily.</p><p><strong>Net Position is Flat:</strong> Their net position (longs minus shorts) is nearly flat, indicating they are well-hedged and not making a large directional bet.</p><p><strong>The Real Risk:</strong> The risk isn't directional; it's a mechanical funding risk. The massive gross inventory creates immense demand for repo funding, making the system vulnerable.</p></td></tr></table>
    """.format(**base64_images),
    # Slide 6: Accumulation Momentum
    """
    <h2 class="slide-title">Dealer Accumulation Momentum</h2>
    <table class="slide-table"><tr><td class="chart-cell"><img src="{accumulation_momentum}" alt="Area chart of dealer inventory accumulation momentum." class="chart-img"></td><td class="text-cell"><p><strong>Measuring the Flow:</strong> This chart compares the 26-week moving average of dealer inventory to the 52-week moving average.</p><p><strong>Rapid Accumulation:</strong> The pink zones highlight periods of rapid accumulation, where the short-term trend exceeds the long-term. We are currently in such a period.</p><p><strong>A Leading Indicator:</strong> This momentum reveals hidden stress. Sustained periods of rapid accumulation, like in 2024-25, have historically preceded market instability.</p></td></tr></table>
    """.format(**base64_images),
    # Slide 7: Stablecoin Demand
    """
    <h2 class="slide-title">The Stablecoin Factor: A New Source of Demand</h2>
    <table class="slide-table"><tr><td class="chart-cell"><img src="{stablecoin_holdings}" alt="Line chart of stablecoin T-Bill holdings." class="chart-img"></td><td class="text-cell"><p><strong>A Concentrated Buyer:</strong> A new, significant buyer has emerged. Stablecoin issuers now hold over $180B in T-Bills, with Tether alone holding $125B.</p><p><strong>Regulatory Arbitrage?</strong> These entities must back their tokens with ultra-safe, liquid assets, creating massive demand for T-bills (typically 90 days or less).</p><p><strong>Fragile Demand:</strong> This demand is a function of the crypto market, not fundamentals. A reversal in crypto or a regulatory crackdown could cause this demand to vanish quickly.</p></td></tr></table>
    """.format(**base64_images),
    # Slide 8: Bills vs Bonds
    """
    <h2 class="slide-title">A Tale of Two Markets: Bills vs. Bonds</h2>
    <table class="slide-table"><tr><td class="chart-cell"><img src="{bills_vs_bonds}" alt="Line chart of T-Bill auction demand." class="chart-img"></td><td class="chart-cell"><img src="{bills_vs_bonds}" alt="Line chart of 10-Year auction demand." class="chart-img"></td></tr><tr><td colspan="2" style="font-size: 15px; text-align: center; padding-top: 15px;"><p>A clear divergence: Demand for short-term T-Bills is robust, while demand for long-term bonds is weakening, shown by falling bid-to-cover ratios.</p></td></tr></table>
    """.format(**base64_images),
    # Slide 9: Supply Surge
    """
    <h2 class="slide-title">The Oncoming Supply Wave</h2>
    <table class="slide-table"><tr><td class="chart-cell"><img src="{supply_surge}" alt="Bar chart of increasing quarterly Treasury issuance." class="chart-img"></td><td class="text-cell"><p><strong>Issuance is Accelerating:</strong> The supply of new Treasury debt is compounding the demand problem. Net issuance is projected to hit $1.5T in the second half of 2025 alone.</p><p><strong>Poor Timing:</strong> This surge in supply is arriving at the precise moment when traditional foreign buyers are retreating and dealers are hitting their balance sheet constraints.</p></td></tr></table>
    """.format(**base64_images),
    # Slide 10: Stress Scenarios
    """
    <h2 class="slide-title">Potential Market Impact Under Stress</h2>
    <table class="slide-table"><tr><td class="chart-cell"><img src="{stress_scenarios}" alt="Bar chart of stress scenario impacts." class="chart-img"></td><td class="text-cell"><p><strong>Quantifying the Risk:</strong> This chart models the potential impact as dealer inventory (a proxy for market stress) increases.</p><p><strong>Moderate Stress ($150B):</strong> If foreign demand continues to decline, 10-year yields could reach 4.75% - 5.00%.</p><p><strong>Severe Stress ($200B+):</strong> A severe scenario could push yields above 5% and lead to a violent bear-steepening of the yield curve.</p></td></tr></table>
    """.format(**base64_images),
    # Slide 11: Stress Dashboard
    """
    <h2 class="slide-title">Treasury Market Stress Dashboard</h2>
    <table style="width:100%; height:80%; border-spacing: 15px; border-collapse: separate;"><tr><td class="chart-cell"><img src="{stress_dashboard}" alt="Chart." class="chart-img"></td><td class="chart-cell"><img src="{new_chart}" alt="Chart." class="chart-img"></td></tr><tr><td class="chart-cell"><img src="{stress_dashboard}" alt="Chart." class="chart-img"></td><td class="chart-cell"><img src="{stress_dashboard}" alt="Chart." class="chart-img"></td></tr></table>
    """.format(**base64_images),
    # Slide 12: Z-Score
    """
    <h2 class="slide-title">Dealer Balance Sheet Stress (Z-Score)</h2>
    <table class="slide-table"><tr><td class="chart-cell"><img src="{z_score}" alt="Line chart of Dealer Position Z-Score." class="chart-img"></td><td class="text-cell"><p><strong>Normalizing the Data:</strong> This chart shows the Z-score of dealer net Treasury positions, measuring how many standard deviations the current position is from its historical average.</p><p><strong>Interpreting the Signal:</strong> Values between +2 and -2 are normal. Spikes beyond indicate significant stress.</p><p><strong>Current Reading:</strong> The current Z-score is well within the normal range, indicating that on a statistically normalized basis, dealer balance sheets are not yet at a breaking point.</p></td></tr></table>
    """.format(**base64_images),
    # Slide 13: Rolling Volatility
    """
    <h2 class="slide-title">Rolling Volatility of Net Positions</h2>
    <table class="slide-table"><tr><td class="chart-cell"><img src="{rolling_volatility}" alt="Line chart of rolling volatility." class="chart-img"></td><td class="text-cell"><p><strong>Measuring Instability:</strong> This chart tracks the rolling 13-week (short-term) and 52-week (long-term) volatility of dealer net positions.</p><p><strong>Spikes in Volatility:</strong> Periods where short-term vol (purple) spikes above long-term vol (grey) indicate market instability.</p><p><strong>Current Reading:</strong> Short-term volatility is elevated and has crossed above the long-term trend, suggesting the *rate of change* in positioning is becoming erratic.</p></td></tr></table>
    """.format(**base64_images),
        # Slide 14: Position Changes (Using 'position_changes' chart)
    """
    <h2 class="slide-title">Dealer Position Changes</h2>
    <table class="slide-table">
        <tr>
            <td class="chart-cell"><img src="{position_changes}" alt="Line chart of dealer position changes." class="chart-img"></td>
            <td class="text-cell">
                <p><strong>Analyzing the Pace of Change:</strong> This chart shows the rate at which dealer net positions are changing.</p>
                <p><strong>Rapid Shifts:</strong> Periods of rapid increase or decrease in net positions can indicate market stress or significant shifts in dealer behavior.</p>
                <p><strong>Current Trend:</strong> Observe the current trend in position changes to understand if dealers are aggressively adding or reducing exposure.</p>
            </td>
        </tr>
    </table>
    """.format(**base64_images),
    # Slide 15: Conclusion
    """
    <h2 class="slide-title">Conclusion & Key Takeaways</h2>
    <div style="font-size: 18px; line-height: 1.8; max-width: 800px; margin: 0 auto;"><ul style="list-style-type: disc; padding-left: 30px; text-align: left;"><li style="margin-bottom: 20px;"><strong>Weakening Foundations:</strong> Multiple data points indicate a structural weakening in the demand for long-term U.S. Treasury debt.</li><li style="margin-bottom: 20px;"><strong>Dealer Constraints are Key:</strong> Primary dealers, the market's backstop, are absorbing massive duration and approaching their regulatory limits.</li><li style="margin-bottom: 20px;"><strong>Supply is a Major Headwind:</strong> An accelerating wave of Treasury supply is set to hit the market just as demand is faltering.</li><li style="margin-bottom: 20px;"><strong>Increased Risk of a "Gap Higher" in Yields:</strong> The combination of these factors increases the probability of a disorderly move higher in long-term interest rates.</li></ul></div>
    """
]

# --- The rest of the script is the same as before ---

final_slides_html = ""
for i, content in enumerate(slides_content):
    active_class = "active" if i == 0 else ""
    final_slides_html += f'<section class="slide {active_class}">{content}</section>\n'

full_html_doc = f"""
<!DOCTYPE html><html lang="en"><head><meta charset="UTF-8"><meta name="viewport" content="width=device-width, initial-scale=1.0"><title>Presentation Preview</title><style>body{{font-family: 'Inter', Arial, sans-serif;background-color: #374151;margin: 0;display: flex;justify-content: center;align-items: center;height: 100vh;}}#presentation-container{{width: 100%;max-width: 1200px;height: 90vh;max-height: 800px;position: relative;box-shadow: 0 10px 25px rgba(0,0,0,0.2);}}.slide{{display: none;width: 100%;height: 100%;padding: 3rem;background-color: #FFFFFF;color: #1F2937;flex-direction: column;justify-content: center;align-items: center;box-sizing: border-box;position: relative;}}.slide.active{{display: flex;}}.slide-title{{font-size: 32px;font-weight: 700;color: #00529B;border-bottom: 2px solid #00529B;padding-bottom: 0.5rem;margin-bottom: 2rem;position: absolute;top: 3rem;left: 3rem;right: 3rem;text-align: left;}}.slide-table{{width: 100%;height: 80%;border-spacing: 20px;border-collapse: separate;margin-top: 4rem;}}.chart-cell{{width: 50%;vertical-align: middle;text-align: center; display: flex; justify-content: center; align-items: center;}}.text-cell{{width: 50%;vertical-align: middle;font-size: 16px;line-height: 1.6;padding: 0 1rem;}}.text-cell p{{margin-bottom: 1.5em;}}.chart-img{{max-width: 100%; max-height: 100%; height: auto; width: auto; object-fit: contain; border: 1px solid #e5e7eb;}}#nav-controls{{position: absolute;bottom: 20px;right: 20px;display: flex;gap: 10px;}}.nav-btn{{background-color: #4B5563;color: white;border: none;border-radius: 50%;width: 50px;height: 50px;font-size: 24px;font-weight: bold;cursor: pointer;box-shadow: 0 4px 6px rgba(0,0,0,0.1);transition: background-color 0.2s;}}.nav-btn:hover{{background-color: #00529B;}}.slide-footer{{position: absolute;bottom: 3rem;left: 3rem;right: 3rem;display: flex;justify-content: space-between;font-size: 14px;color: #6B7280;}}.logo{{font-weight: 700;color: #00529B;}}</style></head><body><div id="presentation-container">{final_slides_html}<div id="slide-counter" class="slide-footer"></div></div><div id="nav-controls"><button id="prevBtn" class="nav-btn">&lt;</button><button id="nextBtn" class="nav-btn">&gt;</button></div><script>const slides = document.querySelectorAll('.slide');const slideCounter = document.getElementById('slide-counter');const totalSlides = slides.length;let currentSlide = 0;function showSlide(index){{currentSlide=(index + totalSlides)%totalSlides;slides.forEach((slide, i) => {{slide.classList.toggle('active', i === currentSlide);}});const footer=slides[currentSlide].querySelector('.slide-footer')||document.createElement('div');if(!slides[currentSlide].contains(footer)){{slides[currentSlide].appendChild(footer);}}footer.innerHTML=`<div class="logo">EquiLend Data & Analytics</div><div>${{currentSlide+1}}/${{totalSlides}}</div>`;}}document.getElementById('nextBtn').addEventListener('click', () => {{showSlide(currentSlide + 1);}});document.getElementById('prevBtn').addEventListener('click', () => {{showSlide(currentSlide - 1);}});document.addEventListener('keydown', (e) => {{if (e.key === 'ArrowRight'){{showSlide(currentSlide + 1);}} else if (e.key === 'ArrowLeft'){{showSlide(currentSlide - 1);}}}});showSlide(0);</script></body></html>
"""

try:
    with open(FILENAME, "w", encoding="utf-8") as f:
        f.write(full_html_doc)
    print(f"✅ Successfully created self-contained presentation: '{FILENAME}'")
except IOError as e:
    print(f"❌ Error writing file: {e}")
    exit()

Handler = http.server.SimpleHTTPRequestHandler
httpd = None
try:
    httpd = socketserver.TCPServer(("", PORT), Handler)
except OSError:
    print(f"❌ Port {PORT} is already in use. Please close the other program or change the PORT number in the script.")
    exit()

url = f"http://localhost:{PORT}/{FILENAME}"
print(f"🚀 Starting server at: {url}")
print("   Navigate with arrow keys or on-screen buttons.")
print("   To stop the server, press Ctrl+C in this terminal.")
webbrowser.open_new_tab(url)

try:
    httpd.serve_forever()
except KeyboardInterrupt:
    print("\n✅ Server stopped.")
    httpd.server_close()

Reading and embedding chart images from local files...
✅ All charts ready.
✅ Successfully created self-contained presentation: 'treasury_foundations_presentation_full.html'
🚀 Starting server at: http://localhost:8000/treasury_foundations_presentation_full.html
   Navigate with arrow keys or on-screen buttons.
   To stop the server, press Ctrl+C in this terminal.
